In [11]:
import re
import typer
import pandas as pd
from typing import Dict
from pathlib import Path
import edgar

# Overview

* Explain EDGAR data, background on why it's useful for Kellogg
* CLI example (screenshot, open terminal and run against sample)
* API example (start up server, go to FastApi doc page, provide example file)
* Code documentation (UML diagram, SQL tables for output)
* Internal code examples can pull directly from actual source

# <center>```edgar2data```: Extracting Information from EDGAR Documents</center>

<br>
<br>
<br>

<center><img src="figures/library.png" width="33%" style='border:5px solid #000000'/></center>


# Example: Insider Trading Data: SEC Forms 3, 4, and 5

Forms 3, 4, and 5 filings are reports submitted to the SEC by investors who may buy or sell shares in companies where they are deemed insiders. The SEC defines an insider as any officer, director or more than 10% shareholder of a publicly traded company.

* https://www.sec.gov/files/forms-3-4-5.pdf
* https://www.sec.gov/Archives/edgar/data/1326190/000101297517000759/xslF345X03/edgar.xml
* https://whalewisdomalpha.com/form-4-insider-trading-analysis/

These filings are publicly available through the [SEC EDGAR website](https://www.sec.gov/edgar/search/)

For example, here is a [Form 4](https://www.sec.gov/Archives/edgar/data/1326190/000101297517000759/)

# ```edgar2data``` uses XML parsing, RegEx, and NLP to extract information

<br>
<br>
<br>

<center><img src="figures/unstrctured-data-types.png" width="80%" style='border:5px solid #000000'/></center>

# <span style="color:purple">Scaling up to Multiple Documents</span>

<br>
<br>
<br>

<center><img src="figures/information_extraction.png" width="100%" style='border:5px solid #000000'/></center>

In [15]:
from edgar.cli import process
help(process)

Help on function process in module edgar.cli:

process(form_type: str = <typer.models.ArgumentInfo object at 0x116d70730>, in_dir: pathlib.Path = <typer.models.ArgumentInfo object at 0x116d70760>, out_dir: Optional[pathlib.Path] = <typer.models.OptionInfo object at 0x116d70ac0>) -> None
    This function expects to be given a directory containing SEC files. Currently supported forms are
    insider trading filings;
    
    (1) form3, (2) form4, (3) form5
    
    The function will process each file and extract the information into a set of 6 flat csv files:
    (1) document_info.csv,
    (2) report_owners.csv,
    (3) signatures.csv,
    (4) footnotes.csv,
    (5) derivatives.csv,git
    (6) nonderivatives.csv.



In [2]:
import re
from typing import Dict

document_fields_header: Dict[str, re.Pattern] = {
    "accession": re.compile(r"^\s*ACCESSION NUMBER:(.+?)$", flags=re.DOTALL | re.MULTILINE),
    "sec_document": re.compile(r"<SEC-DOCUMENT>(.+?):", flags=re.DOTALL | re.MULTILINE),
    "sec_header": re.compile(r"<SEC-DOCUMENT>(.+?):", flags=re.DOTALL | re.MULTILINE),
    "acceptance_datetime": re.compile(r"<ACCEPTANCE-DATETIME>(.+?)$", flags=re.DOTALL | re.MULTILINE)
}
    
for key, val in document_fields_header.items():
    typer.secho(f"key: {key}", fg=typer.colors.WHITE, bg=typer.colors.RED)
    typer.secho(f"val: {val}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

key: accession
val: re.compile('^\\s*ACCESSION NUMBER:(.+?)$', re.MULTILINE|re.DOTALL)
key: sec_document
val: re.compile('<SEC-DOCUMENT>(.+?):', re.MULTILINE|re.DOTALL)
key: sec_header
val: re.compile('<SEC-DOCUMENT>(.+?):', re.MULTILINE|re.DOTALL)
key: acceptance_datetime
val: re.compile('<ACCEPTANCE-DATETIME>(.+?)$', re.MULTILINE|re.DOTALL)


In [3]:
def extract_doc_header_info(f: Path) -> Dict[str, str]:
    text = f.read_text()
    row_dict = {"filename": f.name}
    for field, pat in document_fields_header.items():
        row_dict[field] = None
        match = pat.findall(text)
        if match:
            row_dict[field] = match[0].strip()
        else:
            typer.secho(f"WARNING: {f} does not contain {field}", fg=typer.colors.RED)
    return row_dict

typer.secho(f"{extract_doc_header_info}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

<function extract_doc_header_info at 0x104253160>


In [4]:
from pathlib import Path
import pandas as pd

row_dicts = []
in_dir = Path("../tests/data/form-4/sample/2020")
for f in in_dir.glob("*.txt"):
    typer.secho(f"proccessing file: {f.name}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)
    row_dicts.append(extract_doc_header_info(f))

header_df = pd.DataFrame(row_dicts)
header_df = header_df.set_index("filename")

proccessing file: 1763219_1_0000028412-20-000014.txt
proccessing file: 1656557_3_0001493152-20-018537.txt
proccessing file: 1501695_3_0000950123-20-007937.txt
proccessing file: 897802_3_0001104659-20-084115.txt
proccessing file: 1336745_4_0001213900-20-040637.txt
proccessing file: 1136554_4_0001209191-20-061181.txt
proccessing file: 1632944_3_0001567619-20-015688.txt
proccessing file: 66570_4_0001225208-20-013358.txt
proccessing file: 1109354_3_0001179110-20-009902.txt
proccessing file: 1222848_4_0000746598-20-000179.txt
proccessing file: 1430231_1_0000899243-20-001255.txt
proccessing file: 1491060_3_0001562180-20-006013.txt
proccessing file: 23197_3_0000023197-20-000081.txt
proccessing file: 1034604_2_0001209191-20-036702.txt
proccessing file: 1227500_1_0000899243-20-004098.txt
proccessing file: 1474838_1_0001123292-20-000425.txt
proccessing file: 1555280_4_0001555280-20-000303.txt
proccessing file: 1701114_3_0001104659-20-099652.txt
proccessing file: 1733285_3_0001179110-20-007871.tx

In [5]:
header_df.head(20)

,accession,sec_document,sec_header,acceptance_datetime
filename,,,,
1763219_1_0000028412-20-000014.txt,0000028412-20-000014,0000028412-20-000014.txt,0000028412-20-000014.txt,20200128162633
1656557_3_0001493152-20-018537.txt,0001493152-20-018537,0001493152-20-018537.txt,0001493152-20-018537.txt,20200929160518
1501695_3_0000950123-20-007937.txt,0000950123-20-007937,0000950123-20-007937.txt,0000950123-20-007937.txt,20200807191732
897802_3_0001104659-20-084115.txt,0001104659-20-084115,0001104659-20-084115.txt,0001104659-20-084115.txt,20200716170035
1336745_4_0001213900-20-040637.txt,0001213900-20-040637,0001213900-20-040637.txt,0001213900-20-040637.txt,20201203102720
1136554_4_0001209191-20-061181.txt,0001209191-20-061181,0001209191-20-061181.txt,0001209191-20-061181.txt,20201202163448
1632944_3_0001567619-20-015688.txt,0001567619-20-015688,0001567619-20-015688.txt,0001567619-20-015688.txt,20200818161051
66570_4_0001225208-20-013358.txt,0001225208-20-013358,0001225208-20-013358.txt,0001225208-20-013358.txt,20201112060143
1109354_3_0001179110-20-009902.txt,0001179110-20-009902,0001179110-20-009902.txt,0001179110-20-009902.txt,20200921175501


# <span style="color:purple">Working with XML</span>

https://www.xmlviewer.org/

In [6]:
from typing import Dict

document_fields: Dict[str, str] = {
    "schemaVersion": "schemaVersion",
    "documentType": "documentType",
    "periodOfReport": "periodOfReport",
    "notSubjectToSection16": "notSubjectToSection16",
    "issuerCik": "issuer/issuerCik",
    "issuerName": "issuer/issuerName",
    "issuerTradingSymbol": "issuer/issuerTradingSymbol"
}
    
for key, val in document_fields.items():
    typer.secho(f"key: {key}", fg=typer.colors.WHITE, bg=typer.colors.RED)
    typer.secho(f"val: {val}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

key: schemaVersion
val: schemaVersion
key: documentType
val: documentType
key: periodOfReport
val: periodOfReport
key: notSubjectToSection16
val: notSubjectToSection16
key: issuerCik
val: issuer/issuerCik
key: issuerName
val: issuer/issuerName
key: issuerTradingSymbol
val: issuer/issuerTradingSymbol


In [7]:
from typing import Dict
import xml.etree.ElementTree as ET
import re

def extract_doc_xml_info(f: Path) -> Dict[str, str]:
    text = f.read_text()
    row_dict = {"filename": f.name}
    
    # extract the XML portion of the document using a regex
    xml_pat = re.compile(r"<XML>(.+)</XML>", flags=re.DOTALL)
    match = xml_pat.findall(f.read_text())
    xml_text = match[0].strip()   
    root = ET.fromstring(xml_text)

    # iterate through fields and match on path 
    for field, pat in document_fields.items():
        row_dict[field] = None
        match = root.find(pat)
        if match is not None:
            row_dict[field] = match.text.strip()
        else:
            typer.secho(f"WARNING: {f.name} does not contain {field}", bg=typer.colors.BLACK, fg=typer.colors.WHITE)
    
    return row_dict

typer.secho(f"{extract_doc_xml_info}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

<function extract_doc_xml_info at 0x116caeca0>


In [8]:
import typer
import pandas as pd
from pathlib import Path

row_dicts = []
in_dir = Path("../tests/data/form-4/sample/2020")
for f in in_dir.glob("*.txt"):
    row_dicts.append(extract_doc_xml_info(f))

xml_df = pd.DataFrame(row_dicts)
xml_df = xml_df.set_index("filename")

In [9]:
xml_df.head(20)

,schemaVersion,documentType,periodOfReport,notSubjectToSection16,issuerCik,issuerName,issuerTradingSymbol
filename,,,,,,,
1763219_1_0000028412-20-000014.txt,X0306,4,2020-01-24,None,0000028412,COMERICA INC /NEW/,CMA
1656557_3_0001493152-20-018537.txt,X0306,4,2020-09-28,0,0001076682,"POLARITYTE, INC.",PTE
1501695_3_0000950123-20-007937.txt,X0306,4,2020-08-05,1,0001490892,CoreSite Realty Corp,COR
897802_3_0001104659-20-084115.txt,X0306,4,2020-07-15,0,0000897802,"SPECIAL OPPORTUNITIES FUND, INC.",SPE
1336745_4_0001213900-20-040637.txt,X0306,4,2020-12-01,0,0000350737,OCEAN BIO CHEM INC,OBCI
1136554_4_0001209191-20-061181.txt,X0306,4,2020-12-01,0,0001757898,STERIS plc,STE
1632944_3_0001567619-20-015688.txt,X0306,4,2020-08-15,0,0001564708,NEWS CORP,NWS
66570_4_0001225208-20-013358.txt,X0306,4,2020-11-09,None,0000066570,MSA Safety Inc,MSA
1109354_3_0001179110-20-009902.txt,X0306,4,2020-09-21,0,0001109354,BRUKER CORP,BRKR


In [10]:
# join the results together
df = header_df.join(xml_df)
df.head(10)

,accession,sec_document,sec_header,acceptance_datetime,schemaVersion,documentType,periodOfReport,notSubjectToSection16,issuerCik,issuerName,issuerTradingSymbol
filename,,,,,,,,,,,
1763219_1_0000028412-20-000014.txt,0000028412-20-000014,0000028412-20-000014.txt,0000028412-20-000014.txt,20200128162633,X0306,4,2020-01-24,None,0000028412,COMERICA INC /NEW/,CMA
1656557_3_0001493152-20-018537.txt,0001493152-20-018537,0001493152-20-018537.txt,0001493152-20-018537.txt,20200929160518,X0306,4,2020-09-28,0,0001076682,"POLARITYTE, INC.",PTE
1501695_3_0000950123-20-007937.txt,0000950123-20-007937,0000950123-20-007937.txt,0000950123-20-007937.txt,20200807191732,X0306,4,2020-08-05,1,0001490892,CoreSite Realty Corp,COR
897802_3_0001104659-20-084115.txt,0001104659-20-084115,0001104659-20-084115.txt,0001104659-20-084115.txt,20200716170035,X0306,4,2020-07-15,0,0000897802,"SPECIAL OPPORTUNITIES FUND, INC.",SPE
1336745_4_0001213900-20-040637.txt,0001213900-20-040637,0001213900-20-040637.txt,0001213900-20-040637.txt,20201203102720,X0306,4,2020-12-01,0,0000350737,OCEAN BIO CHEM INC,OBCI
1136554_4_0001209191-20-061181.txt,0001209191-20-061181,0001209191-20-061181.txt,0001209191-20-061181.txt,20201202163448,X0306,4,2020-12-01,0,0001757898,STERIS plc,STE
1632944_3_0001567619-20-015688.txt,0001567619-20-015688,0001567619-20-015688.txt,0001567619-20-015688.txt,20200818161051,X0306,4,2020-08-15,0,0001564708,NEWS CORP,NWS
66570_4_0001225208-20-013358.txt,0001225208-20-013358,0001225208-20-013358.txt,0001225208-20-013358.txt,20201112060143,X0306,4,2020-11-09,None,0000066570,MSA Safety Inc,MSA
1109354_3_0001179110-20-009902.txt,0001179110-20-009902,0001179110-20-009902.txt,0001179110-20-009902.txt,20200921175501,X0306,4,2020-09-21,0,0001109354,BRUKER CORP,BRKR
